# Week 4 - Assignment

Inventory is a class to represent the inventory of a laptop shop. This assignment aims to write queries for searching among the laptops of this shop (stores on a dataset)

## Read dataset

In [60]:
import csv
import re
import pandas as pd

# Define variables to store data.
rows = []     # List to store data rows.
header = []   # List to store the CSV header.

# Specify the CSV file to read.
filename = 'laptops.csv'

# Open the CSV file for reading.
with open(filename, newline='') as file:
    # Create a CSV reader object with a specified delimiter (',').
    csv_reader = csv.reader(file, delimiter=',')
    
    # Read the header row from the CSV and store it in the 'header' variable.
    header = next(csv_reader)
    
    # Loop through the CSV rows, and limit the loop to the first 5 rows (excluding the header).
    for idx, row in enumerate(csv_reader):
        if idx > 4:
            break
        
        # Append the current row to the 'rows' list.
        rows.append(row)

# Print the header to display column names.
print("CSV Header:")
print(header)

# Print the first 5 rows of data.
print("First 5 Rows of Data:")
for row in rows:
    print(row)

CSV Header:
['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
First 5 Rows of Data:
['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339']
['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898']
['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575']
['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537']
['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Co

## Inventory class

In [61]:
def row_price(row):
    """
    Get the price of a laptop from a data row.

    Parameters:
        - row (list): A list containing the laptop's data where the price is stored as the last element.

    Returns:
        - int or float: The price of the laptop as an integer or float.
    """
    return row[-1]


class Inventory:
    def __init__(self, csv_filename):
        """
        Initialize the Inventory object with data from a CSV file.
    
        Parameters:
            - csv_filename (str): The path to the CSV file containing laptop data.
    
        Attributes:
            - header (list): A list containing the column names from the CSV.
            - rows (list): A list of lists, where each sublist represents a laptop's data.
            - id_to_row (dict): A dictionary mapping laptop IDs to their respective data rows.
            - prices (set): A set containing unique laptop prices extracted from the data.
            - rows_by_price (list): A list of laptop data rows sorted by price in ascending order.
        """
        with open(csv_filename, newline='') as file:
            csv_reader = csv.reader(file, delimiter=',')
            self.header = next(csv_reader)
            self.rows = []
            for row in csv_reader:
                self.rows.append(row)
                row[-1] = int(row[-1])
                
            self.id_to_row = {}
            for row in self.rows:
                self.id_to_row[row[0]] = row
                
            self.prices = set()
            for row in self.rows:
                self.prices.add(row[-1])
                
            self.rows_by_price = sorted(self.rows, key=row_price)
    
    def get_laptop_from_id(self, laptop_id):
        """
        Retrieve laptop information based on a unique ID.
    
        Parameters:
            - laptop_id (str): The unique ID of the laptop to retrieve.
    
        Returns:
            - list or None: A list containing the laptop information (if found), or None if the ID is not in the dataset.
        """
        for laptop in self.rows:
            if laptop_id == laptop[0]:
                return laptop
        return None


    def check_promotion_dollars(self, dollars):
        """
        Check if it's possible to purchase a laptop(s) for a given amount of dollars.
    
        Parameters:
            - dollars (int): The amount of dollars available for the purchase.
    
        Returns:
            - bool: True if it's possible to purchase a laptop(s) within the budget, False otherwise.
        """
        # Check if there is a laptop with a price exactly matching the budget.
        for laptop in self.rows:
            if laptop[-1] == dollars:
                return True
        
        # Check if there are two laptops whose prices sum up to the budget.
        for l1 in range(len(self.rows)):
            for l2 in range(l1 + 1, len(self.rows)):
                if self.rows[l1][-1] + self.rows[l2][-1] == dollars:
                    return True
        
        # If no combination of laptops fits the budget, return False.
        return False
    

    def check_promotion_dollars_fast(self, dollars):
        """
        Quickly check if it's possible to purchase a laptop(s) for a given amount of dollars using a more efficient algorithm.
    
        Parameters:
            - dollars (int): The amount of dollars available for the purchase.
    
        Returns:
            - bool: True if it's possible to purchase a laptop(s) within the budget, False otherwise.
        """
        # Create a set to store unique laptop prices.
        seen = set()
        
        # Check if there is a laptop with a price exactly matching the budget.
        if dollars in self.prices:
            return True
        
        # Iterate through the laptops and check if the difference between the budget and a laptop price
        # is in the 'seen' set, indicating a valid combination of laptops.
        for laptop in self.rows:
            if dollars - laptop[-1] in seen:
                return True
            seen.add(laptop[-1])
        
        # If no combination of laptops fits the budget, return False.
        return False
        
    def get_laptops_with_price_within(self, min_price, max_price):
        """
        Retrieve a list of laptops within a specified price range.
    
        Parameters:
            - min_price (int): The minimum price for the laptops to be included.
            - max_price (int): The maximum price for the laptops to be included.
    
        Returns:
            - list: A list of laptops whose prices fall within the specified range.

        Complexity Analysis: for 
            - Big O: O(n)
            - Big Omega: Ω(1)
            - Big Theta: Θ(n)
        """
        # Use list comprehension to filter laptops with prices within the specified range.
        filtered_laptops = [laptop for laptop in self.rows_by_price if min_price <= laptop[-1] <= max_price]
        return filtered_laptops

    def _get_spec_from_ram(self, spec):
        """
        Extract the RAM specification (amount in GB) from a string.

        Parameters:
            - spec (str): The RAM specification string containing a numeric value followed by "GB" or "MB".

        Returns:
            - int: The amount of RAM in gigabytes (GB) as an integer.
        """
        return int(re.sub(r'[a-zA-Z]', '', spec))

    def _get_spec_from_memory(self, string):
        """
        Extract memory specifications (amount and type) from a string.

        Parameters:
            - string (str): The memory specification string in the format "<amount><unit> <type> + <amount><unit> <type> + ..."

        Returns:
            - set: A set of tuples, each containing (amount_storage, type_storage).

        Note:
            - According to dataset, type_storage can be either HDD, Hybrid, Flash Storage or SSD
            
        Examples:
            - "16GB SSD" turns {("16GB","SSD")}
            - "256GB SSD + 1TB HDD" turns {("256GB","SSD"),("1TB","HDD")}
        """
        # Define a RegEx pattern (mask) to match memory specifications in the input string.
        # [A-Za-z]+ : catch storage quantity (1TB, 256GB, etc.)
        # ([A-Za-z\s]+) : catch storage type (TB, HDD, Hybrid, Flash Storage)
        # \s* : storage quantity and storage type are separeted by blank space
        mask = r'(\d+[A-Za-z]+)\s*([A-Za-z\s]+)'
    
        # Use the regular expression to find all matches in the input string and store them in the 'matches' list.
        matches = re.findall(mask, string)
    
        # Initialize an empty set to store the extracted memory specifications as tuples.
        tuples = set()
    
        # Loop through each match found by the regular expression.
        for match in matches:
            # Extract the amount of storage and the type of storage from the match.
            amount_storage = match[0]
            type_storage = match[1].strip()
    
            # Add a tuple containing the amount and type to the 'tuples' set.
            tuples.add((amount_storage, type_storage))
    
        # Return the set of extracted memory specifications as tuples.
        return tuples

    def find_the_cheapest_laptop_with(self, ram, memory):
        """
        Perfoms a linear search to find the cheapest laptop with specific RAM and memory specifications.

        Parameters:
            - ram (str): The desired RAM specification (e.g., "8GB").
            - memory (str): The desired memory specification (e.g., "256GB SSD").

        Returns:
            - list or -1: A list containing the laptop information if found, or -1 if no match is found.

        Complexity Analysis: for n (amount of laptops) and m (length of memory string)
            - Big O: O(n * m)
            - Big Omega: Ω(n * m)
            - Big Theta: Θ(n * m)
        """
        idx = 0
        idx_cheapest = None
        price_cheapest = self.rows_by_price[0][-1]

        desired_ram = self._get_spec_from_ram(ram)
        desired_memory = self._get_spec_from_memory(memory)

        for laptop in self.rows_by_price:
            target_ram = self._get_spec_from_ram(laptop[7])
            target_memory = self._get_spec_from_memory(laptop[8])
            if desired_ram == target_ram and desired_memory == target_memory:
                if laptop[-1] < price_cheapest or idx_cheapest is None:
                    price_cheapest = laptop[-1]
                    idx_cheapest = idx
            idx += 1
            
        if idx_cheapest is None:
            return -1
            
        return self.rows_by_price[idx_cheapest]

## Testing

For better visualization, I've created the helper below

In [62]:
def laptops_formatter(laptops):
    df = pd.DataFrame(laptops, columns=header)
    display(df)

First, we instantiate the class

In [63]:
inventory = Inventory(filename)

Then, we perform the queries

In [69]:
# Laptops whose prices are within 100 and 200
laptops_formatter(inventory.get_laptops_with_price_within(1000,2000))

,Id,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,6676297,HP,EliteBook 840,Notebook,14,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,500GB HDD,Intel HD Graphics 520,Windows 10,1.54kg,1000
1,8747948,Lenovo,ThinkPad T460,Notebook,14,1366x768,Intel Core i5 6200U 2.3GHz,4GB,508GB Hybrid,Intel HD Graphics 520,Windows 7,1.70kg,1002
2,5550925,Dell,Latitude 5580,Notebook,15.6,1366x768,Intel Core i5 7300U 2.6GHz,8GB,500GB HDD,Intel HD Graphics 620,Windows 10,1.9kg,1008
3,3667708,Acer,Aspire F5-573G-510L,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,12GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 950M,Windows 10,2.4kg,1009
4,8017281,Dell,Vostro 5568,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Nvidia GeForce 940MX,Windows 10,2.18kg,1009
...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,9571095,Toshiba,Tecra Z50-C-140,Notebook,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6600U 2.6GHz,16GB,256GB SSD,Nvidia GeForce 930M,Windows 10,2.4kg,1975
480,2623571,Asus,ZenBook Pro,Ultrabook,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD,Nvidia GeForce GTX 1050 Ti,Windows 10,1.8kg,1983
481,8162291,Dell,Precision 3520,Workstation,15.6,Full HD 1920x1080,Intel Xeon E3-1505M V6 3GHz,8GB,64GB Flash Storage + 1TB HDD,Nvidia Quadro M620,Windows 10,2.23kg,1993
482,3043315,HP,Omen 17-w207nv,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,12GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,3.35kg,1999


In [75]:
# Cheapest laptop with 8GB RAM and 256GB SSD
laptops_formatter([inventory.find_the_cheapest_laptop_with('8GB','256GB SSD')])

,Id,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,2618101,Acer,ES1-523-84K7 (A8-7410/8GB/256GB/FHD/W10),Notebook,15.6,Full HD 1920x1080,AMD A8-Series 7410 2.2GHz,8GB,256GB SSD,AMD Radeon R5,Windows 10,2.23kg,469


In [74]:
# Cheapest laptop with 8GB RAM and 128GB SSD + 2TB HDD
laptops_formatter([inventory.find_the_cheapest_laptop_with('8GB','128GB SSD + 2TB HDD')])

,Id,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,3444668,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,128GB SSD + 2TB HDD,AMD Radeon 530,Windows 10,2.02kg,970
